<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Первичный-осмотр-данных" data-toc-modified-id="Первичный-осмотр-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Первичный осмотр данных</a></span></li><li><span><a href="#Очистка-текста" data-toc-modified-id="Очистка-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Очистка текста</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Деление-на-выборки" data-toc-modified-id="Деление-на-выборки-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Деление на выборки</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>TF-IDF</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Метод-опорных-векторов" data-toc-modified-id="Метод-опорных-векторов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Метод опорных векторов</a></span></li><li><span><a href="#Сравнение-моделей" data-toc-modified-id="Сравнение-моделей-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Сравнение моделей</a></span></li><li><span><a href="#Проверка-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-лучшей-модели-на-тестовой-выборке-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Проверка лучшей модели на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Модерация комментариев

Интернет-магазин вводит новый сервис: теперь пользователи имеют возможность редактировать и дополнять описания товаров, аналогично вики-сообществам. Это означает, что клиенты могут предлагать свои правки и комментировать изменения, сделанные другими. Для этого магазину необходим инструмент, который будет автоматически обнаруживать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка
### Первичный осмотр данных

In [1]:
import os
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from nltk.corpus import wordnet, stopwords as nltk_stopwords
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings("ignore")

nltk.download('averaged_perceptron_tagger')

state = 123

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
pth1 = '/datasets/toxic_comments.csv'
pth2 = 'toxic_comments.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1, index_col=[0])
elif os.path.exists(pth2):
    data = pd.read_csv(pth2, index_col=[0])
else:
    print('Something is wrong')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [4]:
data.shape

(159292, 2)

In [5]:
data.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [6]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [7]:
data['toxic'].value_counts(normalize=True) * 100

0    89.838787
1    10.161213
Name: toxic, dtype: float64

Наблюдается значительный дисбаланс классов в пользу отрицательного - его в 9 раз больше. Это стоит учитывать при обучении.

### Очистка текста
Очистим тексты от лишних символов при помощи регулярных выражений.

In [8]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.lower()
    text = text.split()
    return ' '.join(text)

In [9]:
data['text'] = data['text'].apply(clear_text)

In [10]:
data.sample(20)

,text,toxic
12075,the image the main image it s not clean to non...,0
105772,regarding errors in the intro formula p z i k ...,0
59537,if you still think the statement is not reliab...,0
26500,i ve blocked you for forty eight hours for you...,0
114640,returned advice i strongly recommend that you ...,0
85465,okay but i made the page jakewater so how is t...,0
51296,an announcement i would like to make i am now ...,1
135829,berkley high school edit first the edit to the...,0
93126,he deleted my discussion so i delete his i wou...,0
127177,oops i didn t request a new ip from dhcp anywa...,1


### Лемматизация
Проведем лемматизацию.

In [11]:
m = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    token = nltk.word_tokenize(text)
    return ' '.join(m.lemmatize(word, get_wordnet_pos(word)) for word in token)

In [12]:
data['lem_text'] = data['text'].apply(lemmatize)

In [13]:
data.head()

,text,toxic,lem_text
0,explanation why the edits made under my userna...,0,explanation why the edits make under my userna...
1,d aww he matches this background colour i m se...,0,d aww he match this background colour i m seem...
2,hey man i m really not trying to edit war it s...,0,hey man i m really not try to edit war it s ju...
3,more i can t make any real suggestions on impr...,0,more i can t make any real suggestion on impro...
4,you sir are my hero any chance you remember wh...,0,you sir be my hero any chance you remember wha...


### Деление на выборки
Разделим датасет на фичи и таргет, а зачем на выборки - обучающую и тестовую.

In [14]:
features = data['lem_text']
target = data['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target,\
                                                                            test_size=0.15, stratify = target,\
                                                                            random_state=state)
features_train.shape, features_test.shape

((135398,), (23894,))

### TF-IDF
Вычислим TF-IDF.

In [15]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tfidf_train = count_tf_idf.fit_transform(features_train)
tfidf_test = count_tf_idf.transform(features_test)

## Обучение

### Логистическая регрессия

In [16]:
lr = LogisticRegression(random_state = state)

lr_params = {
    'class_weight': (None, 'balanced'),
    'solver': ('liblinear', 'saga'),
    'penalty': ('l2', 'l1'),
    'C': (1, 2, 3, 4)
}

logreg = GridSearchCV(lr, 
                      lr_params, 
                      cv=3, 
                      verbose=2, 
                      scoring='f1',
                      n_jobs = -1)

logreg.fit(tfidf_train, target_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] END C=1, class_weight=None, penalty=l2, solver=liblinear; total time=   7.8s
[CV] END C=1, class_weight=None, penalty=l2, solver=liblinear; total time=   8.1s
[CV] END C=1, class_weight=None, penalty=l2, solver=liblinear; total time=   7.0s
[CV] END ....C=1, class_weight=None, penalty=l2, solver=saga; total time=   2.0s
[CV] END ....C=1, class_weight=None, penalty=l2, solver=saga; total time=   2.0s
[CV] END ....C=1, class_weight=None, penalty=l2, solver=saga; total time=   1.9s
[CV] END C=1, class_weight=None, penalty=l1, solver=liblinear; total time=   1.1s
[CV] END C=1, class_weight=None, penalty=l1, solver=liblinear; total time=   1.2s
[CV] END C=1, class_weight=None, penalty=l1, solver=liblinear; total time=   1.2s
[CV] END ....C=1, class_weight=None, penalty=l1, solver=saga; total time=  23.2s
[CV] END ....C=1, class_weight=None, penalty=l1, solver=saga; total time=  25.1s
[CV] END ....C=1, class_weight=None, penal

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=123), n_jobs=-1,
             param_grid={'C': (1, 2, 3, 4), 'class_weight': (None, 'balanced'),
                         'penalty': ('l2', 'l1'),
                         'solver': ('liblinear', 'saga')},
             scoring='f1', verbose=2)

In [17]:
logreg.best_params_

{'C': 4, 'class_weight': None, 'penalty': 'l1', 'solver': 'saga'}

In [18]:
logreg.best_score_

0.7698832831207335

### Случайный лес

In [19]:
rf = RandomForestClassifier(random_state=state)
forest_params = {
    'max_depth': (7, 10),
    'n_estimators': (25, 50, 100),
    'class_weight': (None, 'balanced')
}

forest = GridSearchCV(rf, 
                      forest_params, 
                      cv=3, 
                      verbose=2, 
                      scoring='f1',
                      n_jobs = -1)

forest.fit(tfidf_train, target_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ....class_weight=None, max_depth=7, n_estimators=25; total time=   4.2s
[CV] END ....class_weight=None, max_depth=7, n_estimators=25; total time=   4.3s
[CV] END ....class_weight=None, max_depth=7, n_estimators=25; total time=   4.3s
[CV] END ....class_weight=None, max_depth=7, n_estimators=50; total time=   8.3s
[CV] END ....class_weight=None, max_depth=7, n_estimators=50; total time=   8.1s
[CV] END ....class_weight=None, max_depth=7, n_estimators=50; total time=   8.2s
[CV] END ...class_weight=None, max_depth=7, n_estimators=100; total time=  16.4s
[CV] END ...class_weight=None, max_depth=7, n_estimators=100; total time=  16.5s
[CV] END ...class_weight=None, max_depth=7, n_estimators=100; total time=  16.2s
[CV] END ...class_weight=None, max_depth=10, n_estimators=25; total time=   6.1s
[CV] END ...class_weight=None, max_depth=10, n_estimators=25; total time=   6.0s
[CV] END ...class_weight=None, max_depth=10, n_e

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=123),
             n_jobs=-1,
             param_grid={'class_weight': (None, 'balanced'),
                         'max_depth': (7, 10), 'n_estimators': (25, 50, 100)},
             scoring='f1', verbose=2)

In [20]:
forest.best_params_

{'class_weight': 'balanced', 'max_depth': 10, 'n_estimators': 100}

In [21]:
forest.best_score_

0.3748750000721845

### Метод опорных векторов

Так как в датасете наблюдается дисбаланс, было принято попробовать еще одну модель обучения с учителей - метод опорных векторов, которую в различных источниках рекомендуют при такой проблеме.

Для SVM была взята линейная модификация, так как согласно документации, она быстрее справится с большим объемом данных. 

In [22]:
sv = LinearSVC(random_state = state)

sv_params = {
    'class_weight': (None, 'balanced'),
    'C': (1, 2, 3, 4)
}

svm = GridSearchCV(sv, 
                   sv_params, 
                   cv=3, 
                   verbose=2, 
                   scoring='f1',
                   n_jobs = -1)

svm.fit(tfidf_train, target_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END .............................C=1, class_weight=None; total time=   0.9s
[CV] END .............................C=1, class_weight=None; total time=   0.9s
[CV] END .............................C=1, class_weight=None; total time=   0.8s
[CV] END .........................C=1, class_weight=balanced; total time=   2.8s
[CV] END .........................C=1, class_weight=balanced; total time=   2.7s
[CV] END .........................C=1, class_weight=balanced; total time=   2.7s
[CV] END .............................C=2, class_weight=None; total time=   1.2s
[CV] END .............................C=2, class_weight=None; total time=   1.2s
[CV] END .............................C=2, class_weight=None; total time=   1.1s
[CV] END .........................C=2, class_weight=balanced; total time=   4.1s
[CV] END .........................C=2, class_weight=balanced; total time=   4.2s
[CV] END .........................C=2, class_weig

GridSearchCV(cv=3, estimator=LinearSVC(random_state=123), n_jobs=-1,
             param_grid={'C': (1, 2, 3, 4), 'class_weight': (None, 'balanced')},
             scoring='f1', verbose=2)

In [23]:
svm.best_params_

{'C': 1, 'class_weight': None}

In [24]:
svm.best_score_

0.7666336787652973

### Сравнение моделей

In [25]:
df = pd.DataFrame([['Logistic Regression', logreg.best_score_], 
                   ['Random Forest', forest.best_score_], 
                   ['Linear SVM', svm.best_score_]], 
                   columns=['Model', 'F1 on CV']).round({'F1 on CV': 3})
df.sort_values(by='F1 on CV', ascending=False)

,Model,F1 on CV
0,Logistic Regression,0.770
2,Linear SVM,0.767
1,Random Forest,0.375


### Проверка лучшей модели на тестовой выборке

In [26]:
pred = logreg.best_estimator_.predict(tfidf_test)
f1_score(target_test, pred)

0.7833814707842701

## Выводы

В нашем распоряжении был набор данных с разметкой о токсичности правок. В них наблюдается заметный дисбаланс классов в сторону отрицательного (нетоксичного текста). 

Была проведена очистка текста от знаков препинания, тегов и прочей лишней информации, затем лемматизация, деление на стратифицированные выборки, расчет TF-IDF и подбор гиперпараметров для трех моделей - логистический регрессии, случайного леса и линейного методо опорных векторов.

Лучший результат на кросс-валидации (f1 = 0.77) показала логистическая со следующими параметрами: <br>
'C': 4, 'class_weight': None, 'penalty': 'l1', 'solver': 'saga'.<br>

На тестовой выборке лучшая модель дала результат f1 = 0.78, что превосходит требуемый по заданию порог. 